In [3]:
import nltk
import pandas as pd
import re
import string
import contractions
import os
from nltk import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords
from nltk.stem import  WordNetLemmatizer
import inflect
# os.chdir('/home/rinkita/courses/NLPSeminar/nlp-quora-kaggle')
# nltk.download('punkt')
# nltk.download('stopwords')
import numpy as np
import collections
from tqdm import tqdm
tqdm.pandas()
from collections import defaultdict
from pprint import pprint

import matplotlib.pyplot as plt

In [4]:
glove_word_embedding_path = "./data/embeddings/glove.840B.300d/glove.840B.300d.txt"
GLOVE_SIZE = 300
embedding_size=64
num_class=2


In [5]:
data = pd.read_csv("./data/train.csv")
x_train=data.question_text
max_document_len=x_train.map(lambda x: len(x.split(' '))).max()


In [6]:
# """Replace contractions in string of text"""
contracted_input = x_train.progress_apply(lambda row:contractions.fix(row))

100%|██████████| 1306122/1306122 [03:53<00:00, 5590.09it/s]


In [7]:


def extract_features(question_texts):  
    word2index = {}
    word2count = {}
    index2word = {0: "pad"}
    n_words = 1  # Count SOS and EOS
    for text in question_texts:
        for word in text:             
            if word not in word2index:
                word2index[word] = n_words
                word2count[word] = 1
                index2word[n_words] = word
                n_words += 1
            else:
                word2count[word] += 1
    return word2index,word2count,index2word,n_words

    

#make all the input text of the same size as size max_length input senetnce, padding with word "PAD"(zero padding)
def make_input(sentence):
    sent_len = len(sentence.split(' '))
    if sent_len < max_document_len:
        padded_sentence = sentence + (max_document_len - len(sentence.split(' '))) * " PAD"
    else:
        padded_sentence = sentence

    return padded_sentence,sent_len


#function to return batch of data
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens,input_metadata):
    
    #shuffling and creating training batch data of batch_size
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    
    #converting sentence to index vector using word2index dictionary
    x = [[input_metadata.word2index[word] for word in data_x[i].lower().split(' ')]for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens



In [8]:
def clean_text(question_text):
    question_text = question_text.translate(str.maketrans('','',string.punctuation))
    question_text = question_text.translate(str.maketrans('','','1234567890'))
    question_text = " ".join(question_text.split())    
    question_text = question_text.split()
    # stop_words = set(stopwords.words('english'))
    # question_text = [w for w in question_text if not w in stop_words and len(w) >= 3]
    question_text = [w for w in question_text if len(w) >= 3]
    question_text = [w.strip('/"“”') for w in question_text ]
    question_text = " ".join(question_text)
    return question_text

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas



In [9]:
from nltk.corpus import wordnet

def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
            
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens

In [10]:
def tokens(text):
    """
    Get all words from corpus
    """
    return re.findall(r'\w+', text)

WORDS = tokens(open("words_alpha.txt", 'r').read())
WORD_COUNTS = collections.Counter(WORDS)


In [11]:
def edits0(word):
    """
    Return all strings that are zero edits away (i.e. the word itself).
    """
    return{word}

def edits1(word):
    """
    Return all strings that are one edits away.
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    def splits(word):
        """
        return a list of all possible pairs
        that the input word is made of
        """
        return [(word[:i], word[i:]) for i in range(len(word)+1)]
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) >1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return(set(deletes + transposes + replaces + inserts))

def edits2(word):
    """
    return all strings that are two edits away.
    """
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words):
    return {w for w in words if w in WORD_COUNTS}


In [12]:
def correct(word):
    candidates = (known(edits0(word)) or
                 known(edits1(word)) or
                 known(edits2(word)) or
                 [word])
    return max(candidates, key=WORD_COUNTS.get)

In [13]:
def correct_match(match):#
    """
    spell-correct word in match, 
    and perserve upper/lower/title case
    """
    word = match.group()
    def case_of(text):
        return(str.upper if text.isupper() else
              str.lower if text.islower() else
              str.title if text.istitle() else
              str)
    return case_of(word)(correct(word.lower()))

def correct_text_generic(text):
    """
    correct all words in text
    """
    return re.sub('[a-zA-Z]+', correct_match, text)

In [14]:

cleaned_text = contracted_input.progress_apply(lambda row:clean_text(row))

word_tokenised = cleaned_text.progress_apply(lambda text:nltk.word_tokenize(text))

lemmatized_words = word_tokenised.progress_apply(lambda word_list:lemmatize_verbs(word_list))

 

100%|██████████| 1306122/1306122 [01:19<00:00, 16391.27it/s]


In [15]:
word2index,word2count,index2word,n_words = extract_features(lemmatized_words)

In [16]:
def get_glove(path_to_glove,word2index_map):
    embedding_weights = {}
    count_all_words = 0
    with open(path_to_glove,'r') as f:  
        for line in f:
            vals = line.split(' ')
            word = str(vals[0])
            if word in word2index_map:                
                count_all_words += 1                                 
                coefs = np.asarray(vals[1:],dtype='float32')
                coefs /= np.linalg.norm(coefs)
                embedding_weights[word] = coefs
    # _WORD_NOT_FOUND = [0.0]* GLOVE_SIZE
    # embedding_weights = defaultdict(lambda: _WORD_NOT_FOUND)
    return embedding_weights

In [17]:

word2embedding_dict = get_glove(glove_word_embedding_path,word2index)


In [18]:
embedding_matrix = np.zeros((n_words,GLOVE_SIZE))

out_of_vocabulary = {}

embeddig_available = 0
for word,index  in word2index.items():
    try:        
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
    except:
        out_of_vocabulary[word] = word2index[word]
        embedding_matrix[index,:] = [0.0]* GLOVE_SIZE
       



In [19]:

print("vocabulary size is {}".format(n_words))
print("no. of words for which no pre-trained embedding found is {}".format(len(out_of_vocabulary)))
print("no. of words for which pre-trained embedding is available{}".format(n_words-len(out_of_vocabulary)))
print("embedding found for {} percentage of vocab".format(((n_words-len(out_of_vocabulary))/n_words)*100))
# out_of_vocabulary


vocabulary size is 259595
no. of words for which no pre-trained embedding found is 100244
no. of words for which pre-trained embedding is available159351
embedding found for 61.384464261638314 percentage of vocab


In [ ]:
corrected_words = []
for k in out_of_vocabulary:
    print(k)
    corrected_words.append(correct_text_generic(k))



feelingslives
storagewill
powersabilities
nodesDOFs
calead
propulsionMS
equation…
Sinatbt
Deutschs
Tepelene
Udacitys
nanodegree
ArchitectureISA
onechild
CPSSPABT
Anizara
HTMLCSS
mathsphysics
conceiveis
Megabites
Redmi
rapedomestic
savegely
ohmagawdcom
gagcom
vendorssuppliers
FIFADUPLEX
Venuas
PakkstaniIndian
RRII
CBRRR
GSXare
mongodump
mongorestore
Ingredio
LRDI
PostTraumatic
species…
Bhashani
Imucet
flappingrvoicing
nasalize
winterwinner
EarthX
websitesforums
DeviantArts
stuter
examinaton
nonoffensive
ChikFilAs
livestudy
Gminor
MarvelDC
promemo
SinAsinB
ramk
Seagos
bandhup
Padmaavat
doubleaction
cleanshot
Swagatham
selectedinvited
millionpost
youS
Mesopotamias
naagmani
YorkiePom
MPlanning
Bakkushan
walmartlabs
adtargeting
nonresidentialcity
multiplehit
nondihydropyridine
SOMAIYA
Commoncom
Ollieco
Colivingcom
simlarity
arichtecture
toothcoloured
Lewenhaupt
Rehnskiöld
THAADs
daughterinlaw
thunderstike
Radon–Nikodym
varnamala
businessmajor
maintanable
IndianAmerican
nonbranded
hivebump
h

engineerscientist
selfdiagnosed
psychopathspotters
Whicho
ClickInvest
SpaceXs
Timsty
collegestheatres
ethereum
MTNLTelephone
cardDads
amaravati
LaWhore
goswell
contenents
momshouse
Textrovert
OLQs
alAdha
Jobstobedone
representme
pregaps
bhawani
supersenses
superbrains
Vivimed
outdoorsin
Samundra
becoome
drivenuclear
mediaon
there…my
selfobsessed
covetness
Tanayra
gwroth
largesized
uwabaki
fashiom
cellswind
adaar
storu
USRussia
fullform
cosxcosx
cosxcosxcosxcosx
genderneutral
USACanadaAustralia
femminists
Airtasker
thrombophob
étatists
wingstips
defineidentity
touristpiligrimagehistoric
isare
laserd
antiChina
Pagari
Pagri
Cplint
Fridabad
Plutarchs
Maliyali
salten
anlamam
alHariri
alkhohol
finger£
underdevelop
technicalIT
wifibluetooth
sexrape
AngloGerman
creepiestscariest
Sambhram
UraniumZirconium
shabari
idiotinchief
nonactors
Qoura
CFIIG
canwill
Kaizala
GDPR
campavan
Mythogy
insultbully
bumpfire
deblur
SDCM
lawHow
secter
waymo
OBOR
WakeUpNow
fruitveg
thPCM
pulmanery
BabeNet
uttarprade

Wellassa
perspctive
Gigafactory
toyshousehold
BringBackOurGirls
believenot
Chhillar
chickensat
limitsis
fxes
sensitiveemotional
Koreaphile
GoDaddycom
inGoDaddycom
selfhosted
dmgamesdll
psI
websitemagazine
Camptlotropus
massachuttes
Ujani
storesin
Maheshwaris
technologyproducts
​​Indus
Hotstar
fhyle
Releasemyad
gamescharacters
tonitrophobia
softwarehardware
Sovereignclass
WHk
proTrump
claculate
percentiles​
metheard
artistsfriendly
scutums
pessit
Yordles
manushi
chhillar
Milometer
BTechMBA
Daesh
ipda
Differenz
Fichteschen
Schellingschen
LabradorGreat
embararrasing
husbandwife
riceburner
Jupyter
Durgakalshakti
heroon
stormfrontorg
IASIPS
expndature
famILY
stanthorpe
drugrelated
BCSCST
xasinwtπ
shitslam
determime
pansexuals
bodybodies
toohigh
gausss
targetswood
AsiansIndians
Wittgensteins
lowcompetition
ethyltrimethylheptane
Mapinduzis
Convectional
frequenct
dogcat
ShopClues
UGCNET
youTsang
avacation
cardiosurgen
economicsbased
mathmathbbHmath
pakhandi
sinA
cosA
cosecAcotA
HyderabadIndia


intimateclose
dhdt
dvdt
PMAY
LDSMormon
students…
triamgle
rolePlaying
bacteriaphages
cosxsinx
editblocked
yonkou
Liiron
Lipolymer
stupidedt
Boomeraction
Twosided
achohal
Wefunder
rightsbased
chrashindo
theBulk
langars
oldfurniture
furnituredealer
girlfriendwife
algorithmsmethods
shallissue
Randyll
لكل
مقام
مقال
demandpull
costpush
NIELIT
nonINTJs
Trumpcare
fastais
Boschs
AustraliaSouth
Spotbit
bigender
required​
ExNRI
cathelistic
httpswwwequiticsin
ALHTIN
CDEBT
Costcocom
sideThis
fopm
Ryzen
DelhiMumbaiBangalore
behaviorfriendships
laporascopy
acceleraton
Googlein
Googlecoin
MTechPhD
Gastroc
privacydata
Galaxysized
DeepCoder
pardubice
singleword
Xioping
indopak
pubbar
metasthasing
MAHMBA
bhabha
PYURIA
Johnnycomelately
Proxmires
extreamism
mathdisplaystyle
zefraczmathrmdzmath
mathzmath
fourdimensional
fivedimensional
gameshopucom
swarg
Takadono
frausmd
USIsrael
Kanziepers
aretopic
Tyrions
Yeadesktop
screenglass
shinchans
actorsactresses
systemtraining
Rajbonshis
prependicular
doorslam
LC

Yaarlo
MakeAWish
Satoshis
budhwar
btcxindia
masturbatingI
facehow
aleves
forawrding
MexicanAmericans
Bhimbra
Asfak
lesbianButch
tofeminine
degreeDiploma
GraphQL
caee
websitesonline
technosoft
mustrbate
irrregular
runoperate
transparenttranslucent
midijobs
PreCalc
univese
GASSES
TMCFree
nonUS
AEISEC
blogwebsitebusiness
LNCT
embarrassingawkward
douchebagstools
douchebaginess
cubbycom
alod
chooda
bitchygene
Cerego
distrubibuted
Ricanswho
…th
…btech
Superdoom
appsite
softwaresite
tips…
tensedHer
Shinoguchi
KozhikodeShillong
CGPAUG
Turkeyball
GeorgiaBall
Hassidc
mandarinfish
kernelsize
CardsCitizenship
lifeGIVE
increasingcontinued
nonpasteurized
JavaMe
fivedigit
Plexcoin
Odoo
Fehrat
Gocem
bhima
respectShe
proportionsjust
crypoto
TESLAs
psychosociopaths
operared
audiomedia
gdown
limdisted
uppercastes
UIUX
astoroyd
paysof
selfreflection
trapdestroy
CodeAddictz
NMIMSS
cyptocurrencies
thgrade
partsthings
Shraddhanand
boughtbottled
DRWill
doard
watersolvent
Travolt
areilly
Actioning
parkerize
em

...

investdouble
Mechnovate
AirtoAir
Jovovichs
Koreastart
Eberytime
filecoin
feelingsthoughts
timetested
plennum
exroommate
Gergerblatzkin
friendfinding
Wikibuys
NASDFINRA
songsend
preIslamic
Shivanis
lythium
múch
marridge
biologybiochemistry
Onlince
itunse
unintendly
DismissiveAvoidant
studyread
availableaffordable
Mtheorys
heliumgaz
includegraphicsh
aftercared
zooligy
ChinaTibet
SotoZen
GreyPhoenix
microslicesof
cànada
Practicemock
TESVI
atttractiveness
HuntJagten
mathaintextGmath
mathleft
paading
bodyskin
asmore
hypercaloric
MarieVictorins
Vaniers
Mumfie
globehostcom
BackSlash
drief
peoplesq
optionopportunities
Rahtee
domainvalidated
JoynerKersees
urbanclap
mathsinradianmath
Dies™️
changegg
rifecarbine
midpole
csatle
alcolizada
detectin
NonSporting
booksarticles
Heiddegger
ereignis
Carneggie
flexipool
copypastas
KarkalaMangalore
claimhttpswwwldsorgscripturesgstemplehouseofthelord
diplycom
being​
Prajina
granuels
abbc
bcCA
CAab
singleuse
toolsguided
tutorialstool
witchunting
ClusterBs
so

binomxnmath
mathxgeq
MobileFiendly
jwRC
PreT
Selenese
profitsharing
WPSK
WilliamsIronheart
aldaily
markdownWYSIWYG
noncarpool
marriageablechildbearing
zuclopenthoxol
Coutus
diphase
isolationdoing
dayobscene
noncoffee
ISCON
Achna
wjile
vaingloryvanity
computeintensive
XRGGZ
redme
shobhit
charaterestics
gaurv
thunderstrom
Mamets
Pinters
multiunits
Raava
spayneuter
Spayneuter
electroscopes
MPaisa
Sugest
WWFs
statesanctioned
annihalate
booknovel
batsmans
Galastop
intodution
lepro
edsl
선배선배님
modernfeminism
forresearch
Popupation
entitiesorganisationscommunities
codeuniform
prataaha
bielections
Kiroromal
untys
friendscops
Ffailure
Kulharis
Jewey
McJew
Jewovitch
Jewski
FrançoisMarie
Arouets
commentsactions
ninesensicall
pilotATCOBS
protozoas
obstructuve
hubbyboyfriend
Pavlakes
beenacquired
scenorio
wishtakers
sfexist
Wuxiaworld
cirporation
perplayer
pigbore
bodywhy
equilization
americocentric
tutorialspoint
PMNarendra
Alipays
Telgram
Whichkind
IADin
RavenSymoné
culturalphilosophical
washedup


earrapes
DengIndira
lawyerbarrister
navyarmy
antiLGBTQ
teleconversation
catalgue
Bbazelon
SEDaily
cwosont
Mandolorians
eletions
JaigaonPhunetsholing
aceking
E·MO·TION
choching
Sheeve
Ravulas
IIFPT
eatworms
eatplants
eathumans
approachContinental
cannongun
KCores
Ribérys
aayd
Hyesun
econbusiness
poormath
scarysad
surroundingssituations
desktoplaptop
Edwiser
catagary
individuvaly
Digibank
phone…reply
GoogleAndroid
schoolissued
Imaprtus
overusedbroken
lassaignes
sequencetosequence
dinocephalian
duyou
youaC
Avenatti
exchangepurse
progressiveslibertariansconservatives
hindiwala
tridosha
MilanOn
highestpsychoticism
awekening
Servil
RGNUL
presidentprime
ministerdictator
like⬇️
nulll
suphuric
countrh
AppAnnie
Saurn
Hugenonts
hearlearn
insidethebowlingball
atomsmolecules
chabahar
illtreatment
alocoholic
Nmat
GoogleApple
transplnt
varietal—easy
by—preferably
addtess
modelize
thinkknow
MInority
McCarranWalter
Hutis
transmitable
AccentureCapgemini
vesicointestinal
Resuts
diifcult
softlythey
SMSCha

ProfaciColombo
ManganoGambino
LucianoGenovese
GaglianoLucchese
YorkUSAThe
nonSlavic
Analyticas
satyashodhak
IITDelhi
sovits
Robin…
ASLeverything
Helcaraxe
Xions
clingyneedy
ISRODRDO
historicalmodern
fastattack
bidrs
AmericanEuropeanJapanSouth
GRUs
soulproprietership
mylawnet
IALM
overallCBI
healthfitness
scienceAI
SGHTG
FullStack
emset
barechest
trriggers
photopcopierscanner
Italianspeaking
pattarn
fanaticallyminded
MilStd
knumbers
Narshima
deptat
singhgad
aptutude
fixedprice
whareas
coldtonsillitis
BrainStation
DYANICS
lovedfelt
Mriza
煮立て
沸騰する
Varyss
Daeneryss
candidacyelection
antiBrown
antiBlack
tagplate
billionyr
nonbaptized
shaoline
clutcher
rupeesmonth
Hexanone
Instabul
anglicise
Gtmetrix
narissses
red​
fravrot
pirsy
AntiPollution
aakashians
katchup
QNETs
faciasm
Trenggalek
neucleus
gutinstincts
IGRUA
radicalist
likesshares
sugeest
interviewexamination
SNIST
njandukalude
nattil
oridavela
redrimmed
SnoopDog
Golbats
Dahiyat
alKheir
enoughgood
autisticAspergers
Floersheim
racydresst

undernourish
Yûgi
Thanios
Dmer
manorialism
Onedollar
assistantssecretary
youndu
taserface
procastinator
Catcherlike
reserveguard
trignonometry
studioclass
studioacademy
selfintroduction
ARPAs
indetail
ShazamDCU
descrt
nofun
lochgyle
pyschopathsociopath
someonebeing
mastertech
fortyyearold
urrea
MLbased
physicsengineering
Mahadji
impeachedassassinated
metastasio
laughgasp
Shangi
Modisterling
biotechVijay
tubell
textbooksonline
httpswwwjpostcomIsraelNewsCultureJapanesePMAbeservedoffensiveshoedessertatNetanyahudinner
diffwerence
மௌன
விரதம்
Hisaichi
Terauchis
answerdo
Yokee
marudhanayagam
Chumlees
atheists…
tnspire
laundrymats
vigomax
subwoffer
honortreat
catination
givenfirst
surnamesfamily
ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZzzzzzzzzzzzzzzZZZZZZZZZZZZZZZZZZZzzzzzzzzzzzzzzzzzzzzzzz


In [1]:
word2index, word2count ,index2word,n_words = extract_features(corrected_words)
word2embedding_dict = get_glove(glove_word_embedding_path,word2index)

NameError: name 'extract_features' is not defined

In [30]:
still_out_of_vocabulary = {}

embeddig_available = 0
for word,index  in word2index.items():
    try:        
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
    except:
        still_out_of_vocabulary[word] = word2index[word]
        embedding_matrix[index,:] = [0.0]* GLOVE_SIZE

NameError: name 'word2index2' is not defined

In [ ]:
print("Out of vocab words")
print(still_out_of_vocabulary)
print("vocabulary size is {}".format(n_words))
print("no. of words for which no pre-trained embedding found is {}".format(len(still_out_of_vocabulary)))
print("no. of words for which pre-trained embedding is available{}".format(n_words-len(still_out_of_vocabulary)))
print("embedding found for {} percentage of vocab".format(((n_words-len(still_out_of_vocabulary))/n_words)*100))
# out_of_vocabulary

In [46]:
#make all the input text of the same size as size max_length input senetnce, padding with word "PAD"(zero padding)
def make_input(sentence):
    sent_len = len(sentence.split(' '))
    if sent_len < max_document_len:
        padded_sentence = sentence + (max_document_len - len(sentence.split(' '))) * " PAD"
    else:
        padded_sentence = sentence

    return padded_sentence,sent_len


#function to return batch of data
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens,input_metadata):
    
    #shuffling and creating training batch data of batch_size
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    
    #converting sentence to index vector using word2index dictionary
    x = [[input_metadata.word2index[word] for word in data_x[i].lower().split(' ')]for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens